# Gray Gas Flux 
### Figure from Chapter 4 of *Principles of Planetary Climate*

In [ ]:
# %load ../../scripts/ch04/GreyGasFlux.py

Computes fluxes and heating rates for the grey gas model. 

The fluxes are computed as a function of $p/p_s$ (given identifier `pps`), given net optical thickness of the atmosphere $\tau_{\infty}$ .

Since the OLR is just the upward flux at $p=0$, this can also be used to do grey gas OLR computations.  Different temperature profiles can be treated by just editing the functions $T(p)$ and $\frac{dT}{dp}(p)$

The moist adiabat function in `phys.py` can be used as well, if one employs the option to create an interpolation function that returns temperature at an arbitrary pressure.


**Known limitations:**  
The optically thick approximation breaks down near the top of the atmosphere, especially with pressure broadening included. This makes plotting difficult. Currently it's handled with a value cutoff, but there's probably a better way.

In [ ]:
import math
import purplebook.ClimateGraphicsMPL as ClimateGraphicsMPL
import purplebook.ClimateUtilities as ClimateUtilities
from purplebook.ClimateUtilities import *
import purplebook.phys as phys

In [ ]:
from astropy.constants import sigma_sb

In [ ]:
sigma_sb.value

Specify temperature as a function of $p/p_s$, so `pps` symbolizes pressure divided by surface pressure.   
$T_{strat}$ and $T_s$ are set as globals.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook', font_scale=1.3)
%config InlineBackend.figure_format='retina'

Define our global variables

In [ ]:
n_layers = 101
p = np.linspace(1/n_layers, 1, num=n_layers)
Rcp = 2.0/7.0
Tstrat = 0.0 #Stratospheric temperature
Ts = 300.0 #Surface air temperature
Tg = 300.0 #Ground temperature

The temperature profile is given simply by the dry adiabatic lapse rate.
$$T(p) = T_{s} \cdot \left( \frac{p}{p_s} \right)^{R/c_p}$$

In [ ]:
def T(pps):
    '''Adiabatic Temperature as a function of pressure'''
    Tadiabat = Ts*(pps)**Rcp
    T_realized = np.maximum(Tstrat,Tadiabat)
    return np.maximum(T_realized, 0.0)

In [ ]:
plt.plot(T(p), p);
plt.ylabel('$p/p_s$'); plt.xlabel('$T(p)\;$(K)')
plt.yscale('log');
plt.text(100, np.min(p)*1.3, 'Top of Atmosphere', color='#bdc3c7')
plt.text(200, np.max(p)/1.1, 'Surface', color='#bdc3c7')
plt.ylim(1, np.min(p)); plt.xlim(0, 300);
plt.title('Pressure-Temperature profile for \nall troposphere Gray Gas model', fontsize=12);

We see a familiar dry adiabatic convection curve.  From Equation 4.37 in the book, we will also need the derivative of tempature with-respect-to-pressure:  
$$\frac{dT}{d(p/ps)}= R/c_p \cdot T_s \cdot (p/p_s)^{(R/c_p -1)}$$

In [ ]:
def dTdp(pps):
    dTadiabat = Rcp*Ts*(pps)**(Rcp-1)
    dT_realized = np.maximum(dTadiabat, 0)
    return dT_realized

In [ ]:
plt.plot(dTdp(p), p);
plt.ylabel('$p/p_s$'); plt.xlabel(r'$\frac{dT}{dp}$', fontsize=18)
plt.yscale('log');

plt.ylim(1, 1e-2);# plt.xlim(0, 300)

Equation 4.10 is the *transmission function*: is the proportion of incident energy flux that is transmitted through a layer of atmosphere extending from $p_1$ to $p_2$.  

Recall the definition of $\tau$ for when $\kappa$ is constant with pressure (i.e. no pressure broadening):

$\tau_{\infty} - \tau = \kappa p/g = \tau_{\infty} p/p_s$

You can verify that $\tau$ is dimensionless through dimensional analysis in cgs units:  

$$\kappa \sim \frac{\text{cm}^2}{\text{gram}}$$ 
$$p \sim \frac{\text{force}}{\text{area}} = \frac{\text{gram}\; \text{s}^{-2}}{\text{cm}}$$ 
$$g \sim \frac{\text{cm}}{\text{s}^{2}}$$  
yielding a cancellation of all units:
$$\frac{\text{cm}^2}{\text{gram}} \cdot \frac{\text{gram}\; \text{s}^{-2}}{\text{cm}} \cdot \frac{\text{s}^{2}}{\text{cm}} \sim \frac{1}{1}$$

In [ ]:
def Trans(tau1,tau2):
    return np.exp(-np.abs(tau1-tau2))

Combining Equation 4.18 and 4.37, we have:

$$ I_+ - I_- = 2 \frac{g \cos{\bar \theta}}{\kappa}(4 \sigma T^3) \frac{dT}{dp}$$

In [ ]:
def integrand(ppsp,tauinf, pps):
    tau1 = tauinf * (1.0 - ppsp)
    tau2 = tauinf * (1.0 - pps)    
    return Trans(tau1,tau2)* 4.0 * sigma_sb.values * T(ppsp)**3 * dTdp(ppsp)

In [ ]:
from scipy.integrate import romberg as romberg_scipy

Compute the upwelling and downwelling flux, $I^+$ and $I^-$, and the heating $H=I^+-I^-$

In [ ]:
def Iplus(pps,tauinf):
    limit = min(1.,pps+10./tauinf)
    quad = romberg(integrand,10)
    tau = tauinf * (1-pps)
    # The term below will be zero if Tg=Ts
    BddTerm = (sigma_sb.value * Tg**4 - sigma_sb.value * Ts**4)*Trans(0.,tau)
    return quad([pps,limit],params,.1) + sigma_sb.value*T(pps)**4 + BddTerm

In [ ]:
def Iminus(pps,tauinf):
    params = Dummy()
    params.pps = pps
    params.tauinf = tauinf
    limit = max(0.,pps-10./tauinf)
    quad = romberg(integrand,10)
    if PressureBroadening:
        tau = tauinf*(1.-pps**2)
    else:
        tau = tauinf*(1.-pps)
    return quad([pps,0.],params,.1)+ phys.sigma*T(pps)**4 - phys.sigma*Tstrat**4*Trans(tau,tauinf)

In [ ]:
#This function returns a curve object containing
#the net upward flux as a function of p/ps (i.e. pressure
#relative to surface pressure) , and also gives
#the optically thick approximation to the net upward flux
#
#Note that the heating in the optically thick approximation becomes
#very large in the upper atmosphere, where, the approximation breaks
#down.  To keep this divergence from messing up the axes of the graph,
#in this routine, the heating rate is cut off at a maximum value, that
#is chosen to be large enough that one can see the divergence between
#the asymptotic and exact (numerical) result.  The flat regions
#of heating in the graph thus have no physical meaning. 
def heatList(tauinf):
    c = Curve()
    c.addCurve(p,'p')
    Ip = [Iplus(pps,tauinf) for pps in p]
    Im = [Iminus(pps,tauinf) for pps in p]
    h = [Ip[i]-Im[i] for i in range(len(p))]
    #**ToDo: Find some better way to keep the optically thick curve from messing up the plots
    if PressureBroadening:
        h1 = [(.5/(pps+1.e-10))*2.*4.*5.67e-8*T(pps)**3*dTdp(pps)/tauinf for pps in p]
    else:
        h1 = [2.*4.*5.67e-8*T(pps)**3*dTdp(pps)/tauinf for pps in p]
    #
    #Cut off the maximum of h1 so it doesn't wreck the plot of h
    maxh = max(h)
    h1 = [min(hh1,2.*maxh) for hh1 in h1]
    #
    c.addCurve(h,'NetFlux','Net Flux, Computed')
    c.addCurve(h1,'NetFluxThick','Net Flux, optically thick approx')
    #Set up options to plot as a profile
    c.switchXY = c.reverseY = True
    #Labels and title
    c.PlotTitle = 'tauInf = %f'%tauinf
    c.Ylabel = 'Flux, W/m**2'
    c.Xlabel = 'Normalized Pressure'
    return(c)

In [ ]:
#Do the plots
c1 = heatList(1.)
c10 = heatList(10.)
c50 = heatList(50.)

In [ ]:
from purplebook.ClimateGraphicsMPL import plot

In [ ]:
plot(c1)
#Note: In the text, we suppressed the plotting of
#the optically thick limit for the tauInf = 1 case, because
#the optically thick approximation is very inaccurate in this
#case and expands the scale of the graph so much it's hard to
#see the variation in the correct result. 
plot(c10)
plot(c50)

#This script can also be used to plot OLR vs. Tg or tauinf,
#as illustated below. The OLR is just Iplus(0,tauinf).
tauList = [.1*i for i in range(1,500)]
OLRList = [Iplus(0.,tauInf) for tauInf in tauList]
cOLR=Curve()
cOLR.PlotTitle ='OLR vs optical depth'
cOLR.addCurve(tauList,'TauInf')
cOLR.addCurve(OLRList,'OLR')
plot(cOLR);

Woooo!  We managed to get the code to run and replicate book figures! Awesome!